# LPBF Thermal Simulation: User Guide

Welcome! This notebook guides you through the `neural_pbf` codebase for Laser Powder Bed Fusion (LPBF) thermal simulation.

## 1. Project Overview
This project simulates the thermal fields in 2D/3D LPBF processes using a Finite Difference Method (FDM) solver implemented in PyTorch.

### Key Components:
- **Physics**: `neural_pbf.physics` (Material properties, Phase changes)
- **Integrator**: `neural_pbf.integrator` (Adaptive time-stepping solver)
- **Scan Strategy**: `neural_pbf.scan` (Laser sources, paths)
- **Visualization**: `neural_pbf.viz` (Artifact generation, XT-Diagrams)
- **Experiments**: `experiments/` (Scripts for long runs)

## 2. Running a Basic Simulation
We use `RunContext` to manage everything: config, state, and artifacts.

In [ ]:
import torch
import shutil
from pathlib import Path
import datetime

from neural_pbf.core.config import SimulationConfig, LengthUnit
from neural_pbf.core.state import SimulationState
from neural_pbf.physics.material import MaterialConfig
from neural_pbf.integrator.stepper import TimeStepper
from neural_pbf.scan.sources import GaussianBeam, GaussianSourceConfig
from neural_pbf.viz.temperature_artifacts import TemperatureArtifactBuilder
from neural_pbf.schemas.artifacts import ArtifactConfig
from neural_pbf.schemas.tracking import TrackingConfig
from neural_pbf.schemas.diagnostics import DiagnosticsConfig
from neural_pbf.schemas.run_meta import RunMeta
from neural_pbf.tracking.run_context import RunContext
from neural_pbf.utils.grid import make_xy_grid

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

## 3. Configuration
Configure the simulation domain and material.

In [ ]:
sim_cfg = SimulationConfig(
    Lx=1.0, Ly=1.0, Nx=64, Ny=64,
    dt_base=1e-5, 
    length_unit=LengthUnit.MILLIMETERS
)

mat_cfg = MaterialConfig(
    k_powder=1.0, k_solid=20.0, k_liquid=20.0,
    cp_base=500.0, rho=7800.0,
    T_solidus=1600.0, T_liquidus=1700.0,
    latent_heat_L=200e3
)

stepper = TimeStepper(sim_cfg, mat_cfg)

## 4. Run Context & Artifacts
Set up where to save results (`artifacts/`) and if we want MLFlow.

In [ ]:
# Toggle ENABLE_MLFLOW to True to log to local mlruns
ENABLE_MLFLOW = False 
run_name = "guide_example_run"

tracking_cfg = TrackingConfig(
    enabled=ENABLE_MLFLOW, 
    backend="mlflow" if ENABLE_MLFLOW else "none",
    run_name=run_name
)
artifact_cfg = ArtifactConfig(enabled=True, png_every_n_steps=20)
diagnostics_cfg = DiagnosticsConfig(enabled=True)

# Proper Run Metadata
run_meta = RunMeta(
    dx=sim_cfg.dx, dy=sim_cfg.dy, dz=0.0, dt=sim_cfg.dt_base,
    grid_shape=[64, 64], length_unit="mm",
    seed=42, device=device, dtype="float32",
    started_at=datetime.datetime.now().isoformat()
)

out_dir = Path("artifacts") / run_name
if out_dir.exists(): shutil.rmtree(out_dir)

viz = TemperatureArtifactBuilder(artifact_cfg)
ctx = RunContext(tracking_cfg, artifact_cfg, diagnostics_cfg, run_meta, out_dir, viz)
ctx.start()

## 5. Execution Loop

In [ ]:
# Initialize State
state = SimulationState(T=torch.full((1, 1, 64, 64), 300.0, device=device).float(), t=0.0)
state.material_mask = torch.zeros_like(state.T, dtype=torch.uint8)

# Laser
source_cfg = GaussianSourceConfig(power=150.0, sigma=0.15e-3)
beam = GaussianBeam(source_cfg)

# Grid (Physical Units)
X, Y = torch.meshgrid(
    torch.linspace(0, sim_cfg.Lx_m, 64, device=device),
    torch.linspace(0, sim_cfg.Ly_m, 64, device=device),
    indexing="xy"
)

# Run for 50 steps
steps = 50
dt_macro = 2e-4  # Large macro step, solver sub-steps adaptively

print(f"Simulating in {out_dir}...")
for i in range(steps):
    # Moving Laser
    x0 = sim_cfg.Lx_m * (0.2 + 0.6 * i / steps)
    y0 = sim_cfg.Ly_m * 0.5
    
    # Heat Source
    Q = beam.intensity(X, Y, None, x0=x0, y0=y0)
    
    # Solve Step
    state = stepper.step_adaptive(state, dt_macro, Q_ext=Q)
    
    # Snapshot (Viz handled by ArtifactBuilder)
    meta = {"t": state.t, "scan_pos": (x0, y0)}
    ctx.maybe_snapshot(i, state, meta)

ctx.end(state)
print("Done!")

## 6. Visualization
View the generated artifacts.

In [ ]:
from IPython.display import Image, display

# XT Diagram
xt_path = out_dir / "plots/png/xt_diagram.png"
if xt_path.exists():
    display(Image(filename=str(xt_path)))
    
# Example Step
step_path = out_dir / f"plots/png/step_{steps//2:06d}_surf.png"
if step_path.exists():
    display(Image(filename=str(step_path)))

## 7. MLFlow (Optional)
If you enabled MLFlow, you can view the run by executing:
```bash
mlflow ui
```
in your terminal and finding the experiment `lpbf-thermal` (or as configured).